In [4]:
%load_ext autoreload
%autoreload 2

# !pip3 install transformers
# !pip3 install torch
!pip3 install accelerate

from utils import load_config, set_environment_variables, num_tokens_from_messages, get_git_files, get_data_files
import sys
import os
config       = load_config('config.yaml')
set_environment_variables(config)
ACCESS_TOKEN = os.environ['GIT_ACCESS_TOKEN']
sys.path.append("/database")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 9.4 MB/s eta 0:00:00


In [2]:
from database import get_documentation_by_url, put_new_repository_documentation, get_file_documentation, delete_documentation_by_url, get_status, get_status_by_file, external_json_to_repo_overview_output, get_all_urls_in_db
from repo_processor import process_file, parallel_process_files, download_and_process_repo_url, get_repo_overview
from OverviewChain import OverviewParser, ConfluenceOverviewChain
from concurrent.futures import ThreadPoolExecutor, as_completed
from langchain.document_loaders import GithubFileLoader
from langchain.callbacks import get_openai_callback
from ConfluenceChain import ConfluenceChain, Parser
from IPython.display import display, Markdown
from pymongo import MongoClient
import matplotlib.pyplot as plt
import networkx as nx
from tqdm import tqdm
import pandas as pd
import nest_asyncio
import requests
import shutil

nest_asyncio.apply()


In [3]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

prompt = pipeline.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
)

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    prompt,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
print(outputs[0]["generated_text"][len(prompt):])


/home/ec2-user/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: Using `low_cpu_mem_usage=True` or a `device_map` requires Accelerate: `pip install accelerate`

In [26]:
repo_url              = "https://github.com/dbhatia00/documentation-generation/" #"https://github.com/Adarsh9616/Electricity_Billing_System"
repo_url              = "https://github.com/thuml/Time-Series-Library"
supported_languages   = ['python', 'java', 'javascript']

# data_dict             = download_and_process_repo_url(repo_url, supported_languages)

In [27]:
url_to_process_repo = f"https://bjxdbdicckttmzhrhnpl342k2q0pcthx.lambda-url.us-east-1.on.aws/?repo_url={repo_url}"
response            = requests.get(url_to_process_repo)
response.json()

'Repository URL successfully enqueued: https://github.com/thuml/Time-Series-Library'

In [ ]:
#TODO: Add code for context lengths greater than 32K tokens

In [35]:

status_url = get_status(repo_url)
# get_status_by_file(repo_url, file_name)
status_url


{'repository_url': 'https://github.com/thuml/Time-Series-Library',
 'overall_status': 'Completed',
 'file_level_status': {'exp/..init...py': 'Completed',
  'data.provider/..init...py': 'Completed',
  'layers/..init...py': 'Completed',
  'models/ETSformer.py': 'Completed',
  'layers/FourierCorrelation.py': 'Completed',
  'layers/Conv.Blocks.py': 'Completed',
  'layers/Crossformer.EncDec.py': 'Completed',
  'layers/Transformer.EncDec.py': 'Completed',
  'exp/exp.basic.py': 'Completed',
  'layers/Autoformer.EncDec.py': 'Completed',
  'run.py': 'Completed',
  'data.provider/m4.py': 'Completed',
  'data.provider/data.factory.py': 'Completed',
  'models/FiLM.py': 'Completed',
  'models/FreTS.py': 'Completed',
  'models/Crossformer.py': 'Completed',
  'layers/StandardNorm.py': 'Completed',
  'exp/exp.anomaly.detection.py': 'Completed',
  'models/FEDformer.py': 'Completed',
  'exp/exp.short.term.forecasting.py': 'Completed',
  'models/Koopa.py': 'Completed',
  'models/..init...py': 'Completed'

In [36]:
docs = get_documentation_by_url(repo_url)



In [37]:

def display_repo_overview_in_md(repo_overview_data):
    display(Markdown(f"## {repo_overview_data.overall_summary}"))
    display(Markdown(f"### Functionalities"))
    display(Markdown(f"#### Key Functionalities"))
    for key, value in repo_overview_data.functionalities.items():
        display(Markdown(f"- **{key}**: {value}"))
    display(Markdown(f"### Project Components"))
    display(Markdown(f"#### Main Components"))
    for key, value in repo_overview_data.project_components.items():
        display(Markdown(f"- **{key}**: {value}"))
    display(Markdown(f"### Database Components"))
    display(Markdown(f"#### Components"))
    for key, value in repo_overview_data.database_components.items():
        display(Markdown(f"- **{key}**: {value}"))
    display(Markdown(f"### Tech Stack"))
    display(Markdown(f"#### Technology Stack"))
    for key, value in repo_overview_data.tech_stack.items():
        display(Markdown(f"- **{key}**: {value}"))
        
display_repo_overview_in_md(docs.repo_overview_data)

## This repository focuses on time series analysis, offering a comprehensive library for tasks such as forecasting, imputation, anomaly detection, and classification. It incorporates advanced neural network models and techniques like Fast Fourier Transform, Inception blocks, and Transformer architectures to process and predict time series data efficiently.

### Functionalities

#### Key Functionalities

- **Forecasting**: Generates future values based on historical time series data.

- **Imputation**: Fills in missing values in time series datasets.

...

#### Technology Stack

- **PyTorch**: The primary deep learning library used for model creation and training.

- **NumPy**: Utilized for numerical operations and transformations.

- **Pandas**: Used for data manipulation and analysis.

- **Matplotlib**: For plotting and visualizing time series data and results.

In [41]:
# class PackageDetail(BaseModel):
#     """
#     Represents details of a package used in a file.

#     Attributes:
#     - usage: How the package is used in the file.
#     - description: Description of the package.
#     """
#     usage: str = Field(description="How the package is used in the file")
#     description: str = Field(description="Description of the package")

# class FunctionDetail(BaseModel):
#     """
#     Represents details of a function defined in a file.

#     Attributes:
#     - name: Name of the function.
#     - description: Description of what the function does.
#     - class_declaration: Declaration of the class to which the function belongs. Should include everything from class to function.
#     - additional_details: Additional information about the function.
#     """
#     name: str = Field(..., description="Name of the function")
#     description: str = Field(..., description="Description of what the function does")
#     class_declaration: str = Field(description="Declaration of the class to which the function belongs. Should include everything from class to function.", default="")
#     additional_details: str = Field(description="Additional information about the function", default="")

# class FileConfluenceOutput(BaseModel):
#     """
#     Represents a file in a repository with its details.

#     Attributes:
#     - file_path: Path of the file.
#     - overall_summary: Overall summary of the file.
#     - packages: Packages used in the file with their details.
#     - functions: Functions defined in the file with their details.
#     """
#     file_path: str = Field(..., description="Path of the file")
#     overall_summary: str = Field(description="Overall summary of the file", default="")
#     packages: dict[str, PackageDetail] = Field(description="Packages used in the file with their details")
#     functions: dict[str, FunctionDetail] = Field(description="Functions defined in the file with their details")
#     confluence_page_id: str = Field(description="Confluence page ID of the file", default="")

def display_file_output_in_md(file_output):
    display(Markdown(f"## {file_output.file_path}"))
    display(Markdown(f"### Overall Summary"))
    display(Markdown(f"{file_output.overall_summary}"))
    display(Markdown(f"### Packages"))
    display(Markdown(f"#### Packages Used"))
    for key, value in file_output.packages.items():
        display(Markdown(f"- **{key}**: {value.usage} - {value.description}"))
    display(Markdown(f"### Functions"))
    display(Markdown(f"#### Functions Defined"))
    for key, value in file_output.functions.items():
        display(Markdown(f"- **{key}**: {value.name} - {value.description}"))
        if value.class_declaration:
            display(Markdown(f"  - **Class Declaration for Function**: {value.class_declaration if value.class_declaration else 'None'}"))
        if value.additional_details:
            display(Markdown(f"  - **Additional Details**: {value.additional_details if value.additional_details else 'None'}"))
            



In [45]:
docs.files
list_doc_files = list(docs.files.values())
display_file_output_in_md(list_doc_files[9])

## layers/Autoformer.EncDec.py

### Overall Summary

The file defines a series of PyTorch modules specifically designed for an Autoformer architecture, which is used for time series forecasting. It includes custom layer normalization, moving average blocks for trend highlighting, series decomposition blocks for extracting seasonal and trend components, and both encoder and decoder layers that incorporate attention mechanisms and convolutional layers for processing time series data.

### Packages

#### Packages Used

...

  - **Class Declaration for Function**: class Encoder(nn.Module):

- **DecoderLayer**: DecoderLayer - Autoformer decoder layer with the progressive decomposition architecture, similar to the encoder but includes cross-attention for processing encoder outputs.

  - **Class Declaration for Function**: class DecoderLayer(nn.Module):

- **Decoder**: Decoder - Autoformer decoder that sequences multiple decoder layers, applies normalization, and optionally a projection layer.

  - **Class Declaration for Function**: class Decoder(nn.Module):

In [14]:
get_all_urls_in_db()

['https://github.com/thuml/Time-Series-Library/',
 'https://github.com/dbhatia00/MonteHall',
 'https://github.com/ajaye2/lumnisfactors-python-api',
 'https://github.com/calmturtle124/react-studio',
 'https://github.com/MadaniKK/api_dataschema',
 'https://github.com/dengyixinliliow/bouncingBall',
 'https://github.com/calmturtle124/YueqiTestRepo',
 'https://github.com/calmturtle124/NewTestRepo',
 'https://github.com/calmturtle124/NewTestRepo2',
 'https://github.com/calmturtle124/NewTestRepo3',
 'https://github.com/dbhatia00/NeuronSimulation',
 'https://github.com/ajaye2/lumnisfactors-python-api/',
 'https://github.com/Adarsh9616/Electricity_Billing_System',
 'https://github.com/dbhatia00/documentation-generation/']

In [7]:
# import boto3
# sqs       = boto3.client('sqs', 'us-east-1')
# queue_url = 'https://sqs.us-east-1.amazonaws.com/018192622412/doc-gen'
# response  = sqs.send_message(
#     QueueUrl=queue_url,
#     MessageBody=str("https://github.com/Adarsh9616/Electricity_Billing_System/")
# )

# """ or """

# https://bjxdbdicckttmzhrhnpl342k2q0pcthx.lambda-url.us-east-1.on.aws/?repo_url=https://github.com/Adarsh9616/Electricity_Billing_System/

In [8]:
# response = sqs.receive_message(
#     QueueUrl=queue_url,
#     MaxNumberOfMessages=1,
#     WaitTimeSeconds=1 # Long polling
# )

# messages = response.get('Messages', [])
# print(messages)
# while len(messages) != 0:
#     response = sqs.receive_message(
#         QueueUrl=queue_url,
#         MaxNumberOfMessages=1,
#         WaitTimeSeconds=1 # Long polling
#     )

#     messages = response.get('Messages', [])
#     print(messages)

In [20]:
# delete_documentation_by_url(repo_url)

DeleteResult({'n': 0, 'electionId': ObjectId('7fffffff000000000000013b'), 'opTime': {'ts': Timestamp(1713840614, 40), 't': 315}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1713840614, 40), 'signature': {'hash': b"\xa3%<\x0c#F\xce\xd2\x848\xf1\x97C\x1fJ\xe4\x96/*'", 'keyId': 7313275603581403138}}, 'operationTime': Timestamp(1713840614, 40)}, acknowledged=True)

In [13]:
# name_of_repo = repo_url.split("/")[-2]
# res          = get_repo_overview(name_of_repo, str(docs.files))